Deep Profiling of Mouse Splenic Architecture with CODEX Multiplexed Imaging

ShortName: Goltsev2028deep

Steps of processing the data from raw to Anndata:

In [ ]:
# 1, Download the raw data from https://data.mendeley.com/datasets/zjnpwh8m5b/1

# imports

In [ ]:
import h5py
import scanpy as sc
import seaborn as sns
import scipy.sparse as sp_sparse
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import anndata as ad
import palettable
import sklearn


# csv to h5ad

In [ ]:
adata_raw = sc.read_csv(data_path+'Suppl.Table2.CODEX_paper_MRLdatasetexpression.csv',dtype='str')

In [ ]:
adata_raw.var_names

In [ ]:
protein_idx_list = list(range(1,30))
protein_idx_list.extend([33,34,35])

In [ ]:
protein_list = adata_raw.var_names[protein_idx_list]

In [ ]:
adata_raw[:,'Global_X'].X

In [ ]:
adata = adata_raw[:,protein_idx_list].copy()
adata.X = adata.X.astype('float')
for var in adata_raw.var_names:
    if var in protein_list:
        continue
    adata.obs[var] = adata_raw[:,var].X

In [ ]:
spatial_mat = np.vstack([adata.obs['X.X'],adata.obs['Y.Y']]).transpose().astype('int')
spatial_tile_mat = np.vstack([adata.obs['Xtile'],adata.obs['Ytile']]).transpose().astype('int')
spatial_global_mat = np.vstack([adata.obs['Global_X'],adata.obs['Global_Y']]).transpose().astype('int')
adata.obsm['spatial'] = spatial_mat
adata.obsm['spatial_tile'] = spatial_tile_mat
adata.obsm['spatial_global'] = spatial_global_mat




In [ ]:
adata_use = adata[adata.obs['sample_Xtile_Ytile']=='MRL-4_X01_Y01']
sc.pl.embedding(adata_use,basis='spatial',
                color=[
                    'Imaging phenotype cluster ID',
                    'niche cluster ID'
                ])

# processing and plotting

In [ ]:
# sc.pp.pca(adata,n_comps=10)
sc.pp.neighbors(adata,n_neighbors=30)
sc.tl.umap(adata,n_components=2)
sc.tl.louvain(adata,resolution=0.5)
sc.tl.paga(adata,groups='louvain')

In [ ]:
sc.pl.umap(adata,color=['louvain',
                        'Imaging phenotype cluster ID',
                        'niche cluster ID'
                       ],ncols=1)

In [ ]:
sc.pl.paga(adata,
           solid_edges='connectivities_tree',
          dashed_edges='connectivities',
           color='Imaging phenotype cluster ID'
#            layout=None
#            single_component=True
#            normalize_to_color=True,
           
          )


In [ ]:
sc.pl.paga_compare(adata,color='louvain',
                   basis = 'X_umap',
                   solid_edges='connectivities_tree',
                  dashed_edges='connectivities',
                   
                
                  )

In [ ]:
adata

In [ ]:
np.unique(adata.obs['sampleID'])

In [ ]:
sample_plot = 'MRL-4'
for sample_plot in np.unique(adata.obs['sampleID']):
    print(sample_plot)
    adata_plot = adata[adata.obs['sampleID']==sample_plot]
    sc.pl.embedding(adata_plot,basis='spatial_global',
                    color=['louvain',
                            'Imaging phenotype cluster ID',
                           'niche cluster ID',
                           'CD45'
                           
                          ],
                    s=20,ncols=1)